# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [18]:
!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [19]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 12.4 MB/s eta 0:00:00


In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

LangChain API Key:··········


We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [26]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31287    0 31287    0     0   171k      0 --:--:-- --:--:-- --:--:--  171k


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70146    0 70146    0     0  58468      0 --:--:--  0:00:01 --:--:-- 58455


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [20]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [27]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [28]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [29]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [30]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 67)

We can save and load our knowledge graphs as follows.

In [31]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 67)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [32]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [33]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

Finally, we can use our `TestSetGenerator` to generate our testset!

In [34]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What advancements has Anthropic made in the de...,[Code may be the best application The ethics o...,Anthropic has produced better-than-GPT-3 class...,single_hop_specifc_query_synthesizer
1,What are some challenges associated with evalu...,[Based Development As a computer scientist and...,Evaluating LLMs is challenging because there a...,single_hop_specifc_query_synthesizer
2,what happen with AI in 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,What are some of the ethical concerns associat...,[easy to follow. The rest of the document incl...,The ethical concerns associated with the use o...,single_hop_specifc_query_synthesizer
4,What advancements did Google introduce with th...,[Prompt driven app generation is a commodity a...,"Google's Gemini 1.5 Pro, released in February ...",single_hop_specifc_query_synthesizer
5,How have advancements in AI models contributed...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI models have significantly c...,multi_hop_abstract_query_synthesizer
6,How has the environmental impact of AI improve...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI has improved du...,multi_hop_abstract_query_synthesizer
7,How has the environmental impact of AI been ad...,[<1-hop>\n\nPrompt driven app generation is a ...,The environmental impact of AI has been addres...,multi_hop_abstract_query_synthesizer
8,How has Claude been utilized in the developmen...,[<1-hop>\n\nThose of us who understand this st...,Claude has been utilized in various ways in th...,multi_hop_specific_query_synthesizer
9,How have advancements in GPT-4 and GPT-4o mode...,[<1-hop>\n\nAnother common technique is to use...,Advancements in GPT-4 and GPT-4o models have s...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [35]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [36]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What Stability AI do with LLMs?,[Code may be the best application The ethics o...,Stability AI is one of the organizations that ...,single_hop_specifc_query_synthesizer
1,Wut is the New York Times doin with the lawsui...,[Based Development As a computer scientist and...,The New York Times has launched a landmark law...,single_hop_specifc_query_synthesizer
2,What happen AI 2023?,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,what's up with DALL-E 3 and how it fits in AI ...,[easy to follow. The rest of the document incl...,The context mentions 'Now add a walrus: Prompt...,single_hop_specifc_query_synthesizer
4,How has increased competition influenced the u...,[<1-hop>\n\nPrompt driven app generation is a ...,Increased competition in 2024 has significantl...,multi_hop_abstract_query_synthesizer
5,How have automated evaluations impacted the de...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations have become crucial in t...,multi_hop_abstract_query_synthesizer
6,How has the universal access to AI models and ...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the universal access to AI models and...",multi_hop_abstract_query_synthesizer
7,How have automated evaluations in AI contribut...,[<1-hop>\n\nPrompt driven app generation is a ...,Automated evaluations in AI have become crucia...,multi_hop_abstract_query_synthesizer
8,What were the major insights about Large Langu...,[<1-hop>\n\neasy to follow. The rest of the do...,"In 2023, Large Language Models (LLMs) were rec...",multi_hop_specific_query_synthesizer
9,How has Claude contributed to advancements in ...,[<1-hop>\n\nThose of us who understand this st...,Claude has played a significant role in the ad...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [37]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [38]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [39]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [42]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [43]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [44]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [46]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [47]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are often described as AI systems that can act on your behalf. However, the term is considered vague and lacks a clear, widely understood definition. There are two main categories of thought regarding agents: one sees them as entities that perform tasks for users (like a travel agent), while the other views them as LLMs (large language models) that utilize tools in a loop to solve problems. The challenge of gullibility in AI systems complicates their ability to make meaningful decisions, leading to skepticism about their utility.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [48]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [49]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        }
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        }
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [50]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'weary-join-12' at:
https://smith.langchain.com/o/ec0ab543-3d20-5b47-b548-e29646706b91/datasets/3a5a1a1a-541b-462a-a6f6-2fd751380a0b/compare?selectedSessions=2bffbdb2-79e8-42c1-b6d9-faa5559630db




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the efficiency and multimodal capabil...,The context mentions that OpenAI's GPT-4o is p...,None,GPT-4o is significantly more efficient and cos...,0,0,0,5.041388,ca48dcba-46a2-4ddb-8b11-d1cc68c59cd7,743bcf5b-fccd-4306-b4e0-b26e4cb5de7b
1,How does the challenge of gullibility in LLMs ...,The challenge of gullibility in LLMs is seen a...,None,The challenge of gullibility in LLMs is a sign...,1,1,1,3.282705,b8ce4f70-6e9c-447e-841d-dd7be3e7eb4a,60292724-284b-4dce-a647-eec35a128473
2,How has Claude contributed to advancements in ...,I don't know.,None,Claude has played a significant role in the ad...,0,0,0,2.115450,a51404f2-0b86-4114-a2fb-dac3bec59539,42cf391e-3e44-4cb4-88a9-19b020a4c099
3,What were the major insights about Large Langu...,"In 2023, major insights about Large Language M...",None,"In 2023, Large Language Models (LLMs) were rec...",1,1,0,6.464991,37524fe5-db8a-4c20-b0ad-b48799c46f3e,2b136359-e0b2-46cc-b221-40e45ba388e9
4,How have automated evaluations in AI contribut...,Automated evaluations in AI have become essent...,None,Automated evaluations in AI have become crucia...,1,0,0,2.515541,ddf6c96d-327e-43d5-94ae-d56beece429a,9bea100e-ceb7-47c1-92a3-f5c1f3efbcf4
5,How has the universal access to AI models and ...,The universal access to AI models and increase...,None,"In 2024, the universal access to AI models and...",1,0,0,2.981631,33961482-4a7c-45e6-8cf3-f9d1e2bd9e55,2e3e796a-d38f-4ea5-87b6-004f2829c8d0
6,How have automated evaluations impacted the de...,Automated evaluations have become a crucial sk...,None,Automated evaluations have become crucial in t...,1,0,1,3.058455,e952f97e-200f-4698-a47e-f212fbcfce38,9a237991-e3fa-4751-b46e-2e5d4e4d5a63
7,How has increased competition influenced the u...,Increased competition has led to a crash in LL...,None,Increased competition in 2024 has significantl...,1,0,0,2.312575,1a43ad7c-2253-4bc2-8334-a7efdde4def3,1a2376d0-80b1-44fa-a5ad-8cc97c26a187
8,what's up with DALL-E 3 and how it fits in AI ...,I don't know.,None,The context mentions 'Now add a walrus: Prompt...,0,0,0,1.447458,5cc60263-f813-438d-8fd3-e36ed94c5717,6e9d0aa3-e17a-4650-924f-00273a2e71bb
9,What happen AI 2023?,2023 was the breakthrough year for Large Langu...,None,2023 was the breakthrough year for Large Langu...,1,1,1,2.352015,619d2041-2204-4490-9542-7c01c89d3b32,2309edd4-36b6-4d01-8be7-6531e62fd562


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [51]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [52]:
rag_documents = docs

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [54]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [55]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [56]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [57]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [58]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents are these super vague AI systems people keep talking about that are supposed to act on your behalf. Some folks think of them like travel agents, while others see them as LLMs (language models) using tools to solve problems. The term is tossed around a lot, but there’s no clear definition, which makes it kinda frustrating. They’re like this elusive dream, still in prototype phase, but not quite ready for prime time—mostly because of the whole gullibility issue. Until we tackle that, agents might just be a concept we keep chasing!'

Finally, we can evaluate the new chain on the same test set!

In [59]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'stupendous-whip-97' at:
https://smith.langchain.com/o/ec0ab543-3d20-5b47-b548-e29646706b91/datasets/3a5a1a1a-541b-462a-a6f6-2fd751380a0b/compare?selectedSessions=3efe7ebe-11d2-4153-bcd1-7d3cbe62d1ef




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the efficiency and multimodal capabil...,"Yo, check it out! The efficiency of GPT-4o is ...",None,GPT-4o is significantly more efficient and cos...,1,0,1,4.970672,ca48dcba-46a2-4ddb-8b11-d1cc68c59cd7,cb29ae77-f551-4861-9f03-ba815280668a
1,How does the challenge of gullibility in LLMs ...,"Alright, so here's the scoop: the whole challe...",None,The challenge of gullibility in LLMs is a sign...,1,1,1,8.595514,b8ce4f70-6e9c-447e-841d-dd7be3e7eb4a,31afa004-afb7-48a4-a7a0-24c690a9d856
2,How has Claude contributed to advancements in ...,"Yo, Claude is totally shaking things up in the...",None,Claude has played a significant role in the ad...,1,0,1,3.959817,a51404f2-0b86-4114-a2fb-dac3bec59539,449ad9a1-f28b-4833-a250-648e7f781720
3,What were the major insights about Large Langu...,"Yo, in 2023, we learned that Large Language Mo...",None,"In 2023, Large Language Models (LLMs) were rec...",0,0,1,4.020684,37524fe5-db8a-4c20-b0ad-b48799c46f3e,dde1d781-581b-47a0-92be-326cec7fc2a9
4,How have automated evaluations in AI contribut...,"Yo, automated evaluations in AI are like the s...",None,Automated evaluations in AI have become crucia...,1,1,1,4.197048,ddf6c96d-327e-43d5-94ae-d56beece429a,12e40298-e3d0-4cea-8221-b661b7e2d448
5,How has the universal access to AI models and ...,"Yo, the scene has totally flipped in 2024! Wit...",None,"In 2024, the universal access to AI models and...",1,0,1,4.593320,33961482-4a7c-45e6-8cf3-f9d1e2bd9e55,a910fdb7-03dd-4304-a86a-2c2e67133cd0
6,How have automated evaluations impacted the de...,"Yo, automated evaluations are the secret sauce...",None,Automated evaluations have become crucial in t...,1,1,1,3.648504,e952f97e-200f-4698-a47e-f212fbcfce38,e1709809-aec4-47d8-b181-4644f3859364
7,How has increased competition influenced the u...,"Yo, check it! Increased competition has totall...",None,Increased competition in 2024 has significantl...,1,0,1,3.426422,1a43ad7c-2253-4bc2-8334-a7efdde4def3,f99e8238-5954-4452-93b8-10b4afa56157
8,what's up with DALL-E 3 and how it fits in AI ...,"Yo, DALL-E 3 is all about that prompt engineer...",None,The context mentions 'Now add a walrus: Prompt...,1,1,1,3.461005,5cc60263-f813-438d-8fd3-e36ed94c5717,8ad3ef9e-9363-4a5d-a91f-d8ace7ea61ec
9,What happen AI 2023?,"Yo, 2023 was the year Large Language Models (L...",None,2023 was the breakthrough year for Large Langu...,1,1,1,2.949245,619d2041-2204-4490-9542-7c01c89d3b32,bf7787cf-7a6a-4396-9c99-6a6db74c5fb6


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.